# Merging, Drop, and Variable Renaming
This is the initial pre-processing before doing the variable analysis.

### Algorithm
Step 1 - Merge the data  
Step 2 - Remove the columns with more than 30% missingness  
Step 3 - Create a dictionary  
Step 4 - Lookup against dictionary  
Step 5 - Split into test/train data  

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from pathlib import Path
import os, re

In [2]:
# convert XPT files to pandas dataframe
def xpt_to_df(file_path):
    df = pd.read_sas(file_path, format='xport', encoding='utf-8')
    return df

##### Step 1 - Merge the Data

In [3]:
root = Path("RAW/DATA")
xpt_files = list(root.rglob("*.xpt"))

P_GHB_path = 'RAW/DATA/laboratory_data/P_GHB.xpt'
P_GHB = pd.read_sas(P_GHB_path, format='xport', encoding = 'utf-8')

base = P_GHB.copy()
base['SEQN'] = pd.to_numeric(base['SEQN']).astype('Int64')
base_idxed = base.set_index('SEQN')

used_cols = set(base_idxed.columns) # Tracking used column names for collisions

fileskip_list = [
    # Dietary Data
    'P_DR1IFF.xpt',
    'P_DR2IFF.xpt',
    'P_DR1TOT.xpt',
    'P_DR2TOT.xpt',
    'P_DRXFCD.xpt',
    'DSBI.xpt',
    'DSII.xpt',
    'DSPI.xpt',
    'P_DS1IDS.xpt',
    'P_DS2IDS.xpt',
    'P_DS1TOT.xpt',
    'P_DS2TOT.xpt',
    'P_DSQIDS.xpt',
    'P_DSQTOT.xpt',

    # Audiometry Sensor Data
    'P_AUXAR.xpt',
    'P_AUXTYM.xpt',
    'P_AUXWBR.xpt',

    # Medication Survey Data
    'P_RXQ_RX.xpt'
]

dfs = []  # all other dfs, already indexed by SEQN
log_path = Path("LOG/log_merge.txt")

with open(log_path, "w") as log:
    for p in xpt_files:
        filename = os.path.basename(p)
        stem = Path(filename).stem
        print(f"Loading: {filename}")

        if filename in fileskip_list:
            print(f"\tSkipping {filename} (manually excluded)")
            continue

        # Read with fallback encoding
        try:
            df = pd.read_sas(p, format="xport", encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_sas(p, format="xport", encoding="cp1252")

        if 'SEQN' not in df.columns:
            print(f"\tSkipping {filename}, missing SEQN")
            continue

        
        df['SEQN'] = pd.to_numeric(df['SEQN']).astype('Int64')

        dup_counts = df['SEQN'].value_counts()
        dup_counts = dup_counts[dup_counts > 1]
        if not dup_counts.empty:
            # print(f"\tSkipping {filename}. Found {dup_counts.size} duplicated SEQN values")
            msg = f"Skipping {filename}. Found {dup_counts.size} duplicated SEQN values\n"
            print(f"\t{msg.strip()}")
            log.write(msg)
            continue
        df = df.set_index('SEQN')
        
        # # Renaming only colliding columns (prefixing with filename)
        # rename_mapper = {}
        # for col in df.columns:
        #     if col in used_cols:
        #         rename_mapper[col] = f"{filename}_{col}"
        # if rename_mapper:
        #     df = df.rename(columns=rename_mapper)

        df = df.rename(columns={col: f"{stem}__{col}" for col in df.columns})

        # Check for duplicate columns across files
        overlapping = [col for col in df.columns if col in used_cols]
        if overlapping:
            msg = f"{filename}: Found {len(overlapping)} duplicate columns: {overlapping}\n"
            print(f"\t{msg.strip()}")
            log.write(msg)

        used_cols.update(df.columns) # Update used set

        dfs.append(df)

# ---- One big combine ----
# Concatenate all other files horizontally (align on SEQN once)
others = pd.concat(dfs, axis=1, copy=False)

# Single join to the base
df_01_merged = base_idxed.join(others, how='left')
df_01_merged = df_01_merged.reset_index()

print(f"\nMerging complete. Duplicate column log saved to {log_path.resolve()}")

Loading: P_DEMO.xpt
Loading: DSBI.xpt
	Skipping DSBI.xpt (manually excluded)
Loading: DSII.xpt
	Skipping DSII.xpt (manually excluded)
Loading: DSPI.xpt
	Skipping DSPI.xpt (manually excluded)
Loading: P_DR1IFF.xpt
	Skipping P_DR1IFF.xpt (manually excluded)
Loading: P_DR1TOT.xpt
	Skipping P_DR1TOT.xpt (manually excluded)
Loading: P_DR2IFF.xpt
	Skipping P_DR2IFF.xpt (manually excluded)
Loading: P_DR2TOT.xpt
	Skipping P_DR2TOT.xpt (manually excluded)
Loading: P_DRXFCD.xpt
	Skipping P_DRXFCD.xpt (manually excluded)
Loading: P_DS1IDS.xpt
	Skipping P_DS1IDS.xpt (manually excluded)
Loading: P_DS1TOT.xpt
	Skipping P_DS1TOT.xpt (manually excluded)
Loading: P_DS2IDS.xpt
	Skipping P_DS2IDS.xpt (manually excluded)
Loading: P_DS2TOT.xpt
	Skipping P_DS2TOT.xpt (manually excluded)
Loading: P_DSQIDS.xpt
	Skipping P_DSQIDS.xpt (manually excluded)
Loading: P_DSQTOT.xpt
	Skipping P_DSQTOT.xpt (manually excluded)
Loading: P_AUX.xpt
Loading: P_AUXAR.xpt
	Skipping P_AUXAR.xpt (manually excluded)
Loading: P_A

##### Step 2 - Remove the columns with more than 30% missingness  

In [4]:
# Step 2a - Remove columns with more than 30% missingness
num_rows = df_01_merged.shape[0]
threshold = 0.3 * num_rows

cols_to_drop = df_01_merged.columns[df_01_merged.isnull().sum() > threshold]
df_02_dropped = df_01_merged.drop(columns=cols_to_drop)

# Step 2b - Remove rows with no target variable
target_col = 'P_GHB__LBXGH'
before_rows = df_02_dropped.shape[0]
df_02_dropped = df_02_dropped.dropna(subset=[target_col])
after_rows = df_02_dropped.shape[0]
rows_dropped = before_rows - after_rows

# Logging the cleaning steps
log_path = Path("LOG/log_cleaning.txt")
with open(log_path, "w") as log:
    log.write("==== Data Cleaning Log ====\n\n")

    # Log columns dropped
    log.write(f"Columns dropped (>30% missingness): {len(cols_to_drop)}\n")
    if len(cols_to_drop) > 0:
        log.write("\n".join(cols_to_drop))
        log.write("\n\n")
    else:
        log.write("None\n\n")

    # Log rows dropped
    log.write(f"Rows dropped with missing target variable ({target_col}): {rows_dropped}\n")

print(f"Dropped {len(cols_to_drop)} columns with >30% missingness.")
print(f"Dropped {rows_dropped} rows with missing target variable ({target_col}).")
print(f"Cleaning log saved to {log_path.resolve()}")

Dropped 1107 columns with >30% missingness.
Dropped 672 rows with missing target variable (P_GHB__LBXGH).
Cleaning log saved to C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\LOG\log_cleaning.txt


##### Step 3 - Create a dictionary  

In [5]:
import pandas as pd
from pathlib import Path
import re

# Folder with all codebooks
codebook_root = Path("RAW/CODEBOOKS")

# Helper to clean symbols
def clean_text(s):
    s = "" if pd.isna(s) else str(s)
    s = re.sub(r"[^A-Za-z0-9_]+", "_", s)
    return re.sub(r"_+", "_", s).strip("_")

# Collect all codebook files
merged = []
for f in codebook_root.rglob("*_codebook.csv"):
    df = pd.read_csv(f)
    df.columns = [c.lower().replace(" ", "_") for c in df.columns]
    df = df.rename(columns={"variable_name": "var", "variable": "var",
                            "sas_label": "label", "label": "label"})
    if "var" not in df or "label" not in df:
        continue
    data_file = Path(f).stem.replace("_codebook", "")
    df["variable_name"] = df["var"].apply(clean_text)
    df["sas_label"] = df["label"].apply(clean_text)
    df["variable_label"] = df["variable_name"] + "_" + df["sas_label"]
    df["data_file"] = data_file
    merged.append(df[["data_file", "variable_name", "sas_label", "variable_label"]])

# Combine and save
if merged:
    out = pd.concat(merged, ignore_index=True)
    output_path = Path("TABLES/dictionary.csv")
    output_path.parent.mkdir(parents=True, exist_ok=True)
    out.to_csv(output_path, index=False)
    print(f"Saved dictionary to: {output_path.resolve()}")
else:
    print("No valid codebooks found.")

Saved dictionary to: C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\TABLES\dictionary.csv


##### Step 4 - Lookup against dictionary  

In [8]:
dict_path = "TABLES/dictionary.csv"
df_dict = pd.read_csv(dict_path)

# Mapping: {old_name: new_name}
# The old name is data_file + "__" + variable_name (same as in merged data)
df_dict["old_name"] = df_dict["data_file"] + "__" + df_dict["variable_name"]
rename_map = dict(zip(df_dict["old_name"], df_dict["variable_label"]))

# Apply renaming
df_03_renamed = df_02_dropped.rename(columns=rename_map)


# Save dtypes
output_path = Path("INPUTS/CSV/df_03_renamed_dtypes.csv")
df_03_renamed.dtypes.to_csv(output_path, index=True)
print(f"Saved dataframe dtypes to: {output_path.resolve()}")

# export
print(f"Renamed {len(rename_map)} columns using dictionary.")
output_path = Path("INPUTS/CSV/df_03_renamed.csv")
df_03_renamed.to_csv(output_path, index=False)
print(f"Saved renamed dataframe to: {output_path.resolve()}")

Saved dataframe dtypes to: C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\INPUTS\CSV\df_03_renamed_dtypes.csv
Renamed 3738 columns using dictionary.
Saved renamed dataframe to: C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\INPUTS\CSV\df_03_renamed.csv


##### Step 5 - Flag and set table output to clearly categorical data  

In [ ]:
# df = df_03_renamed.copy()

# flags = []
# dup_counter = {}  # to label duplicate column names in the output

# for i, col in enumerate(df.columns):
#     # Make an output-safe name for duplicates
#     k = dup_counter.get(col, 0)
#     out_name = col if k == 0 else f"{col}__dup{k}"
#     dup_counter[col] = k + 1

#     # Always select by index -> guaranteed 1-D Series
#     s = df.iloc[:, i].dropna()

#     # Empty -> treat as numeric (False) to avoid all-True
#     if s.empty:
#         flags.append((out_name, False))
#         continue

#     # Try convert to numeric on the *values* only
#     s_num = pd.to_numeric(pd.Series(s.values), errors="coerce")
#     numeric_ratio = s_num.notna().mean()

#     # Mostly non-numeric -> categorical
#     if numeric_ratio < 0.9:
#         flags.append((out_name, True))
#         continue

#     # Numeric path
#     s_num = s_num.dropna()

#     # Any decimals -> numeric
#     if (s_num % 1 != 0).any():
#         flags.append((out_name, False))
#         continue

#     # Integer-like -> categorical if few codes
#     nunique = s_num.nunique()
#     if nunique <= 20 or (nunique / len(s_num)) < 0.01:
#         flags.append((out_name, True))
#     else:
#         flags.append((out_name, False))

# # Build and save flag table
# df_flag = pd.DataFrame(flags, columns=["var_name", "IS_CATEGORICAL"])
# out_path = Path("TABLES/variable_flags.csv")
# out_path.parent.mkdir(parents=True, exist_ok=True)
# df_flag.to_csv(out_path, index=False)

# print(f"Saved variable flags to: {out_path.resolve()}")
# print(df_flag["IS_CATEGORICAL"].value_counts())


Saved variable flags to: C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\TABLES\variable_flags.csv
IS_CATEGORICAL
         501
P_MCQ     20
Name: count, dtype: int64


In [ ]:
# # Step 5 – Flag categorical vs numeric and include file_name (from dictionary)
# import pandas as pd
# import numpy as np
# from pathlib import Path

# df = df_03_renamed.copy()

# # Load dictionary: must contain columns ["variable_label","data_file"]
# dict_path = Path("TABLES/dictionary.csv")
# df_dict = pd.read_csv(dict_path)

# # Map each final column name (variable_label) -> data_file
# label_to_file = dict(zip(df_dict["variable_label"], df_dict["data_file"]))

# flags = []
# dup_counter = {}

# for i, col in enumerate(df.columns):
#     # Unique output name for duplicates
#     k = dup_counter.get(col, 0)
#     out_name = col if k == 0 else f"{col}__dup{k}"
#     dup_counter[col] = k + 1

#     # Lookup file_name from dictionary (blank if not found)
#     file_name = label_to_file.get(col, "")

#     # Always select by column index to ensure 1-D Series
#     s = df.iloc[:, i].dropna()

#     if s.empty:
#         flags.append((out_name, file_name, False))
#         continue

#     # Try numeric conversion
#     s_num = pd.to_numeric(pd.Series(s.values), errors="coerce")
#     numeric_ratio = s_num.notna().mean()

#     # Mostly non-numeric -> categorical
#     if numeric_ratio < 0.9:
#         flags.append((out_name, file_name, True))
#         continue

#     s_num = s_num.dropna()

#     # Any decimals -> numeric
#     if (s_num % 1 != 0).any():
#         flags.append((out_name, file_name, False))
#         continue

#     # Integer-like -> categorical if few codes
#     nunique = s_num.nunique()
#     if nunique <= 20 or (nunique / len(s_num)) < 0.01:
#         flags.append((out_name, file_name, True))
#     else:
#         flags.append((out_name, file_name, False))

# # Save result
# df_flag = pd.DataFrame(flags, columns=["var_name", "file_name", "IS_CATEGORICAL"])
# out_path = Path("TABLES/variable_flags.csv")
# out_path.parent.mkdir(parents=True, exist_ok=True)
# df_flag.to_csv(out_path, index=False)

# print(f"Saved variable flags to: {out_path.resolve()}")
# print(df_flag["IS_CATEGORICAL"].value_counts())


Saved variable flags to: C:\Users\victo\Documents\CSE 881\PROJECT\CSE881_PROJ\TABLES\variable_flags.csv
IS_CATEGORICAL
True     326
False    195
Name: count, dtype: int64


Step 6 - Split into test/train data  